# ASE genetic algorithm tutorial

Have you remembered to `export PYTHONPATH=/users/ext/nomad20/.local/lib/python3.4/site-packages:$PYTHONPATH`?

In this tutorial we will try to determine the convex hull of a binary metal slab using the genetic algorithm (GA) and EMT potential in ASE.

The convex hull can be used to check whether a certain composition is stable or it will decompose into mixed phases of the neighboring stable compositions. We will use a (111) slab to represent a close packed surface, the method can easily be extended for use in other systems, e.g. bulk, nanoparticle, ... We choose a rather small atomic structure with 24 atoms in the unit cell, in a binary system the number of different atomic distributions for a single composition is determined by the binomial coefficient $\frac{N!}{n_A!n_B!}$, where $N$ is the total number of atoms in the slab, $n_A$ and $n_B$ is the number of A and B atoms respectively. This number rises combinatorially towards the 1:1 composition and in total there exists 16.8 million different atomic distributions for the 24 atom slab (without taking symmetry into account which will reduce the number significantly). A number of this size warrants a search method other than brute force, here we use a GA.


## Initializing the GA

We will start by selecting which metals we are going to study. The EMT potential in ASE supports the following metals: Al, Ni, Cu, Pd, Ag, Pt and Au.  Let's select Cu and Pt


In [ ]:
metals = ['Cu', 'Pt']


Next we will determine the lattice constants of these elements.

In [ ]:
from __future__ import print_function
from ase.lattice.cubic import FaceCenteredCubic
from ase.eos import EquationOfState
from ase.calculators.emt import EMT
calc = EMT(asap_cutoff=True)

def get_lattice_constant(m):
    # Make an fcc structure with the experimental lattice constant
    atoms = FaceCenteredCubic(m)
    # Attach the calculator to the Atoms object
    atoms.set_calculator(calc)
    # The unit cell vectors has the same length as the lattice constant
    a = atoms.cell[0][0]

    # Calculate energies at different volumes using the equation of state
    eps = 0.05
    volumes = (a * np.linspace(1 - eps, 1 + eps, 9))**3
    energies = []
    for v in volumes:
        atoms.set_cell([v**(1. / 3)] * 3, scale_atoms=True)
        energies.append(atoms.get_potential_energy())

    # Fit volumes and energies to a polynomium and return the 
    eos = EquationOfState(volumes, energies)
    v1, e1, B = eos.fit()
    return v1**(1 / 3)

lattice_constants = dict((m, get_lattice_constant(m)) for m in metals)
print('Lattice constants:', lattice_constants)


Next we will calculate the reference energies of pure metal slabs needed to get the mixing energies of mixed slabs.

In [ ]:
from ase.build import fcc111
from ase.ga import set_raw_score

pure_slabs = []
reference_energies = {}
print('References:')
for m in metals:
    slab = fcc111(m, size=(2, 4, 3), a=lattice_constants[m],
                  vacuum=5, orthogonal=True)
    slab.set_calculator(calc)

    e = slab.get_potential_energy()
    e_per_atom = e / len(slab)
    reference_energies[m] = e_per_atom
    print('{0} = {1:.3f} eV/atom'.format(m, e_per_atom))

    set_raw_score(slab, 0.0)
    pure_slabs.append(slab)


Let's look at the slabs to see if they make sense.

In [ ]:
from ase.visualize import view

view(pure_slabs[0], viewer='x3d')

## Running the GA

Now we need to set up the genetic algorithm. Run the cell below if *and only if* you need to restart the algorithm.

In [ ]:
import os
if os.path.isfile('hull.db'):
    os.remove('hull.db')

We initialize a database file that is going to contain all the candidates. We choose a population size large enough so that the entire composition range will be represented in the population.


In [ ]:
from ase.ga.data import PrepareDB
pop_size = 2 * len(slab)
initdb = PrepareDB('hull.db', population_size=pop_size,
                   reference_energies=reference_energies, metals=metals,
                   lattice_constants=lattice_constants)


We add the pure slabs to the database as relaxed because we have already set the raw_score.

In [ ]:
for slab in pure_slabs:
    initdb.add_relaxed_candidate(slab,
                                 atoms_string=''.join(slab.get_chemical_symbols()))


Now we create the rest of the candidates for the initial population. The composition and atomic distribution are chosen randomly making sure that at least one of each metal element is present. For the lattice constant we employ Vegard's law (interpolation).

In [ ]:
import random

from ase.build import fcc111

def get_avg_lattice_constant(syms):
    a = 0.
    for m in set(syms):
        a += syms.count(m) * lattice_constants[m]
    return a / len(syms)

def get_random_candidate(max_number_of_element):
    nA = random.randint(0, max_number_of_element)
    nB = max_number_of_element - nA
    symbols = [metals[0]] * nA + [metals[1]] * nB + metals

    # Making a generic slab with the correct lattice constant
    slab = fcc111('X', size=(2, 4, 3),
                  a=get_avg_lattice_constant(symbols),
                  vacuum=5, orthogonal=True)

    # Setting the symbols and randomizing the order
    slab.set_chemical_symbols(symbols)
    random.shuffle(slab.numbers)
    
    return slab

for i in range(pop_size - 2):
    slab = get_random_candidate(2 * 4 * 3 - 2)
    
    # Add these candidates as unrelaxed, we will evaluate them later
    atoms_string = ''.join(slab.get_chemical_symbols())
    initdb.add_unrelaxed_candidate(slab, atoms_string=atoms_string)


Now the database has been initialized, we will connect to the database file with the `DataConnection` class.

In [ ]:
from ase.ga.data import DataConnection
rundb = DataConnection('hull.db')

Next we will evaluate the starting population. The only hard requirement of the evaluation is that the raw_score needs to be set.
To make comparisons between different compositions we define the mixing or excess energy by

$$E_\text{mixing} = E_{AB} - \frac{E_A \cdot n_A}{N} - \frac{E_B \cdot n_B}{N}$$

where $E_\text{AB}$ is the energy of the mixed slab, $E_A$ and $E_B$ are the energies of the pure A and B slabs respectively. With the function below we get the mixing energy from an Atoms object.


In [ ]:
def get_mixing_energy(atoms):
    # Set the correct cell size from the lattice constant
    new_a = get_avg_lattice_constant(atoms.get_chemical_symbols())
    # Use the orthogonal fcc cell to find the current lattice constant
    current_a = atoms.cell[0][0] / np.sqrt(2)
    atoms.set_cell(atoms.cell * new_a / current_a, scale_atoms=True)

    # Calculate the energy
    atoms.set_calculator(calc)
    e = atoms.get_potential_energy()

    # Subtract contributions from the pure element references
    # to get the mixing energy
    syms = atoms.get_chemical_symbols()
    for m in set(syms):
        e -= syms.count(m) * reference_energies[m]
    return e

Negative mixing energy means more stable than the pure slabs. The optimization always progress towards larger raw score, so we take the negative mixing energy as the raw score.


In [ ]:
from ase.ga import set_raw_score

while rundb.get_number_of_unrelaxed_candidates() > 0:
    a = rundb.get_an_unrelaxed_candidate()
    set_raw_score(a, -get_mixing_energy(a))
    rundb.add_relaxed_step(a)


Next we choose the type of population we want. Normally the population evolves around a single global optimum, here since we would like to determine the optimal arrangement of atoms for all compositions we need to use a population that can evolve different niches of the population at once. `ase.ga.population.RankFitnessPopulation` works by grouping candidates according to a variable (composition in this case) and then ranking candidates within each group. This means that the fittest candidate in each group is given equal fitness and has the same probability for being selected for procreation. This means that the entire convex hull is mapped out contrary to just the candidates with lowest mixing energies. This "all in one" approach is more efficient than running each composition individually since the chemical ordering is similar for different compositions.

In [ ]:
from ase.ga.population import RankFitnessPopulation

def get_comp(atoms):
    return atoms.get_chemical_formula()

pop = RankFitnessPopulation(data_connection=rundb,
                            population_size=pop_size,
                            variable_function=get_comp)
pop.update()

Now we need to specify which operators to use for evolving the population. We will use the typical operators adjusted to work on slabs: `CutSpliceSlabCrossover` cuts two slabs in a random plane and put halves from different original slabs (parents) together to form a new slab (offspring). This Deaven and Ho style crossover is able to refine the population by passing on favorable traits from parents to offspring. `RandomSlabPermutation` permutes two atoms of different type in the slab keeping the same composition. `RandomCompositionMutation` changes the composition of the slab.

The prepended integer specifies the relative number of times the operator is performed.


In [ ]:
from ase.ga.offspring_creator import OperationSelector
from ase.ga.slab_operators import (CutSpliceSlabCrossover,
                                   RandomSlabPermutation,
                                   RandomCompositionMutation)

oclist = [(3, CutSpliceSlabCrossover()),
          (1, RandomSlabPermutation()),
          (1, RandomCompositionMutation())
          ]
operation_selector = OperationSelector(*zip(*oclist))


Now we are ready to start the iterative part of the algorithm. 

In [ ]:
num_gens = 1

for _ in range(num_gens):
    new_generation = []
    for _ in range(pop_size):
        # Select parents for a new candidate
        parents = pop.get_two_candidates()

        # Select an operator and use it
        op = operation_selector.get_operator()
        offspring, desc = op.get_new_individual(parents)
        # An operator could return None if an offspring cannot be formed
        # by the chosen parents
        if offspring is None:
            continue

        set_raw_score(offspring, -get_mixing_energy(offspring))
        new_generation.append(offspring)

    # We add a full relaxed generation at once, this is faster than adding
    # one at a time
    rundb.add_more_relaxed_candidates(new_generation)

    # update the population to allow new candidates to enter
    pop.update()



To analyze the current population we plot the convex hull. If there are too many points on the plot try changing `only_label_simplices` to `only_plot_simplices`.

In [ ]:
from ase.phasediagram import PhaseDiagram
from ase.db import connect

db = connect('hull.db')

refs = []
dcts = list(db.select('relaxed=1'))
for dct in dcts:
    refs.append((dct.formula, -dct.raw_score))

pd = PhaseDiagram(refs)
pd.plot(only_label_simplices=True)


The convex hull is *probably* not that well described yet. Try to run the algorithm for several more generations by changing num_gens to a larger number or running the cell several times.

We can also investigate the evaluated candidates in the database with the `ase.db` tool.

In [ ]:
from ase.db import connect
from ase.ga import get_raw_score
from collections import defaultdict

db = connect('hull.db')

db_candidates = defaultdict(int)
for row in db.select(sort='-raw_score', limit=pop_size):
    db_candidates[row.formula] += 1
print('Diversity of best candidates in database:', db_candidates.items())

pop_candidates = defaultdict(int)
for atoms in pop.get_current_population():
    formula = atoms.get_chemical_formula()
    pop_candidates[formula] += 1
print('Diversity of candidates in population:', pop_candidates.items())
    

## Customizing the GA

So far we have a working algorithm but it is quite naive, let us make some extensions for increasing efficiency.


### Exact duplicate identification

Evaluating identical candidates is a risk when they are created by the operators, so in order not to waste computational resources it is important to implement a check for whether an identical calculation has been performed. The list of elements in the candidate determines the structure completely, thus we can use that as a measure to see if an identical candidate has been evaluated.

We need to add a key representing the list of chemical symbols for each candidate in the database. We call it `atoms_string`, note that it was already assigned to the initial candidates, thus we only need to use the cell below for the iterative part of the GA instead of the previous one.

In [ ]:
num_gens = 1

for _ in range(num_gens):
    new_generation = []
    for _ in range(pop_size):
        duplicate = True
        while duplicate:
            # Select parents for a new candidate
            parents = pop.get_two_candidates()

            # Select an operator and use it
            op = operation_selector.get_operator()
            offspring, desc = op.get_new_individual(parents)
            # An operator could return None if an offspring cannot be formed
            # by the chosen parents
            if offspring is None:
                continue
            
            atoms_string = ''.join(offspring.get_chemical_symbols())
            duplicate = rundb.is_duplicate(atoms_string=atoms_string)
            
        set_raw_score(offspring, -get_mixing_energy(offspring))
        new_generation.append(offspring)

    # We add a full relaxed generation at once, this is faster than adding
    # one at a time
    rundb.add_more_relaxed_candidates(new_generation)

    # update the population to allow new candidates to enter
    pop.update()


### Symmetric duplicate identification

A further enhancement would be to determine if a structure that is not identical in positions but instead symmetrically identical has been evaluated. For this we need a metric with which to characterize a structure, a symmetry tolerant fingerprint. There are many ways to achieve this and we will use a very simple average number of nearest neighbors, defined as:
$$\text{NN}_\text{avg} = [\frac{\#\text{Cu-Cu}}{N_{\text{Cu}}} , \frac{\#\text{Cu-Pt}}{N_{\text{Cu}}}, \frac{\#\text{Pt-Cu}}{N_{\text{Pt}}}, \frac{\#\text{Pt-Pt}}{N_{\text{Pt}}}]$$

where $\#\text{Cu-Cu}$ is the number of Cu - Cu nearest neighbors and $N_\text{Cu}$ is the total number of Cu atoms in the slab. This check can be performed at two points; either just after candidate creation before evaluation or after evaluation before potential inclusion into the population. The latter method is well suited for situations where the evaluation will induce a change in the candidate e.g. by structural relaxation. We will use the former method here.

The nearest neighbor average is put in `candidate.info['key_value_pairs']` as a string rounded off to two decimal points. *Note* this accuracy is fitting for this size slab, but need testing for other systems.

First we start by restarting the GA and making a new initial population.

In [ ]:
if os.path.isfile('hull.db'):
    os.remove('hull.db')

pop_size = 2 * len(slab)
initdb = PrepareDB('hull.db', population_size=pop_size,
                   reference_energies=reference_energies, metals=metals,
                   lattice_constants=lattice_constants)

for slab in pure_slabs:
    initdb.add_relaxed_candidate(slab,
                                 atoms_string=''.join(slab.get_chemical_symbols()))

for i in range(pop_size - 2):
    slab = get_random_candidate(2 * 4 * 3 - 2)
    
    # Add these candidates as unrelaxed, we will evaluate them later
    atoms_string = ''.join(slab.get_chemical_symbols())
    initdb.add_unrelaxed_candidate(slab, atoms_string=atoms_string)
    

Now we need to add the nearest neighbor average when evaluating the initial candidates as well.


In [ ]:
from ase.ga.utilities import get_nnmat_string
from ase.ga import set_raw_score

rundb = DataConnection('hull.db')

# Evaluating the starting population is changed to
while rundb.get_number_of_unrelaxed_candidates() > 0:
    a = rundb.get_an_unrelaxed_candidate()
    # The following line is added
    a.info['key_value_pairs']['nnmat_string'] = get_nnmat_string(a, 2, True)
    set_raw_score(a, -get_mixing_energy(a))
    rundb.add_relaxed_step(a)


We will add a comparator to the population instance so that no two candidates with the same nearest neighbor average are present in the population.

In [ ]:
from ase.ga.standard_comparators import StringComparator

pop = RankFitnessPopulation(data_connection=rundb,
                            population_size=pop_size,
                            variable_function=get_comp,
                            comparator=StringComparator('nnmat_string'))
pop.update()


Then we run the iteractive part of the algorithm.

In [ ]:
num_gens = 1

for _ in range(num_gens):
    new_generation = []
    for _ in range(pop_size):
        duplicate = True
        while duplicate:
            # Select parents for a new candidate
            parents = pop.get_two_candidates()

            # Select an operator and use it
            op = operation_selector.get_operator()
            offspring, desc = op.get_new_individual(parents)
            # An operator could return None if an offspring cannot be formed
            # by the chosen parents
            if offspring is None:
                continue
            
            atoms_string = ''.join(offspring.get_chemical_symbols())
            duplicate = rundb.is_duplicate(atoms_string=offspring.info['key_value_pairs']['atoms_string'])
            
        # If the offspring is not a duplicate we must add the nnmat_string to
        # the key_value_pairs
        nnmat_string = get_nnmat_string(offspring, 2, True)
        offspring.info['key_value_pairs']['nnmat_string'] = nnmat_string
        set_raw_score(offspring, -get_mixing_energy(offspring))
        new_generation.append(offspring)

    # We add a full relaxed generation at once, this is faster than adding
    # one at a time
    rundb.add_more_relaxed_candidates(new_generation)

    # update the population to allow new candidates to enter
    pop.update()


### Problem specific mutation operators

Sometimes it is necessary to introduce operators that force the GA to investigate certain areas of the phase space. The `SymmetrySlabPermutation` permutes the atoms in the slab to yield a more symmetric offspring. *Note* this requires [spglib](https://atztogo.github.io/spglib/) to be installed. Try it by adding the operator to the list of operators.

In [ ]:
from ase.ga.slab_operators import SymmetrySlabPermutation

oclist = [(3, CutSpliceSlabCrossover()),
          (1, RandomSlabPermutation()),
          (1, RandomCompositionMutation()),
          (1, SymmetrySlabPermutation()),
          ]
operation_selector = OperationSelector(*zip(*oclist))


## GA vs random search

Let us check whether our approach using the GA is actually more efficient than a brute force search. A complete screening has been performed with the fast EMT implementation in [ASAP](https://wiki.fysik.dtu.dk/asap/asap). The cell below goes through the candidates evaluated by the GA and notes how many evaluations were needed before the lowest possible mixing energy were found. This is plotted against an estimate of needed evaluations for a random search (a mathematical expression also used by [Jain et al.](https://link.springer.com/article/10.1007%2Fs10853-013-7448-9)). This is of course a silly comparison because no one would use random search for this type of optimization problem.

In [ ]:
from ase.db import connect
import matplotlib.pyplot as plt

top_number = 24
search_space = 16777216

def random_search(number_of_candidates):
    return (number_of_candidates * (search_space + 1)) / (top_number + 1)

# Lowest mixing energy for all compositions found by brute force search using ASAP
mixing_energies = {"Pt24": 0.000000, "Cu24": 0.000000, "Cu10Pt14": -2.247559, "Cu11Pt13": -2.335093, "Cu12Pt12": -2.455589, "Cu13Pt11": -2.388796, "Cu14Pt10": -2.364747, "Cu15Pt9": -2.258926, "Cu16Pt8": -2.171208, "Cu17Pt7": -2.073299, "Cu18Pt6": -2.007277, "Cu19Pt5": -1.720920, "Cu20Pt4": -1.426294, "Cu21Pt3": -1.111488, "Cu22Pt2": -0.799612, "Cu23Pt": -0.417315, "Cu2Pt22": -0.735420, "Cu3Pt21": -0.985260, "Cu4Pt20": -1.267542, "Cu5Pt19": -1.503187, "Cu6Pt18": -1.714879, "Cu7Pt17": -1.877877, "Cu8Pt16": -2.086304, "Cu9Pt15": -2.156783, "CuPt23": -0.348717}

eps = 1e-4
db = connect('hull.db')
found = set()
i = 0
ga_plot = [(0, 0)]
for row in db.select('relaxed=1'):
    i += 1
    s = row.formula
    if s not in found:
        # Rounding errors will prevent us from perfect agreement with ASAP values
        if -row.raw_score < mixing_energies[s] + eps:
            found.add(s)
            print(i, s)
            ga_plot.append((i, len(found)))

x, y = zip(*ga_plot)
plt.plot(x, y, 'bo-', label='GA')

y = np.array(range(top_number + 1))
plt.plot(random_search(y), y, 'k-', label='Random search')
plt.xlabel('Evaluations performed')
plt.ylabel('Candidates found')
plt.show()



This is the final convex hull with only the lowest mixing energy plotted for each composition. ![PtCu convex hull](intro-figure.png)

## Things to try

Try to run the algorithm again with the duplicate detection and extra operators to see if the number of evaluated structures goes down, but remember that the GA is non-deterministic so in order to compare efficiency of parameters one has to do statistics of many runs. The GA could also be run pool-based instead of generational, try to add each candidate to the database individually as they are evaluated and update the population after each addition, this should lower the total number of evaluations required to determine the convex hull. Population size also plays a role in the convergence of the algorithm

Another extension to the tutorial could be to only allow different elements in the top three layers of a thicker slab. This would replicate a surface alloy.
